In [1]:
import pandas as pd
import numpy as np
from ax_experiments_functions import rdc_loss_setup, rdc_read_glch_data, rdc_label_to_params
from ax_utils import gpei_method, build_experiment, initialize_experiment,df_to_trials,initialize_experiment_with_trials
from ax.modelbridge.registry import Models

In [2]:
data_csv_path = "/home/lucas/Documents/perceptronac/scripts/tradeoffs/bpp-mse-psnr-loss-flops-params_bmshj2018"+\
"-factorized_10000-epochs_D-3-4_L-2e-2-1e-2-5e-3_N-32-64-96-128-160-192-224_M-32-64-96-128-160-192-224-256-288-320.csv"

In [3]:
lmbda="2e-2"
mult=100
weights=[1,float(lmbda)*(255**2),1/(1e6 * mult)]
lambdas=[lmbda]
complexity_axis="params"

In [4]:
seed= 1234
n_init = 6
n_batch=34

In [5]:
search_space, optimization_config = rdc_loss_setup(data_csv_path,weights,lambdas,complexity_axis)

In [23]:
gpei_list = gpei_method(search_space,optimization_config,seed,n_init,n_batch)

In [7]:
glch_csv_path = "/home/lucas/Documents/perceptronac/scripts/tradeoffs/gho_results/"+\
"glch1D_weights_1e+00_1e+03_1e-08_lambdas_2e-2_bpp_loss_vs_mse_loss_vs_params_start_left_history.csv"

In [8]:
glch_df = rdc_read_glch_data(glch_csv_path)

In [9]:
glch_exp = build_experiment(search_space,optimization_config)

In [10]:
trials = []
for r,c in glch_df.iterrows():
    trials.append({
        "input":rdc_label_to_params(r),
        "output": { "rdc_loss":{"mean":sum(weights * c[["bpp_loss","mse_loss",complexity_axis]].values), "sem": 0} }
    })

In [11]:
initialize_experiment_with_trials(glch_exp,trials)

In [24]:
glch_exp.fetch_data()

In [13]:
objective_means = np.array([[trial.objective_mean for trial in glch_exp.trials.values()]])
glch_list = np.minimum.accumulate(objective_means, axis=1).reshape(-1).tolist()

In [22]:
pd.DataFrame({"glch_list": glch_list,
"gpei_list": gpei_list}).plot()